<a href="https://colab.research.google.com/github/pakdaniel/MECS-4995-AML-Group-24/blob/main/check_memory_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pandas -U

# https://github.com/huggingface/autotrain-advanced/issues/265

In [1]:
%pip install polars -U

In [1]:
import pandas as pd
import numpy as np
import os
import glob

assert pd.__version__ == "2.1.2"

In [1]:
sh = """for y in {2013..2022}
do
    wget --no-check-certificate "https://gaftp.epa.gov/castnet/CASTNET_Outgoing/data/metdata_$y.zip"
    mkdir -p metdata
    unzip metdata_$y.zip -d metdata
    rm metdata_$y.zip
    sleep 1
done
"""
with open("metdata.sh", "w") as file:
  file.write(sh)

In [ ]:
!bash metdata.sh

In [3]:
files = glob.glob(os.path.join("metdata" , "metdata_20[1-2][0-9].csv"))
res = 0
for i in files:
  info = os.stat(i)
  res += info.st_size
print(f"Size of files is {res/1024**2:.2f} mb")

Size of files is 874.88 mb


In [4]:
# This uses 4.4 to 4.7 gb in RAM

df_list = []
dtypes = {"SHELTER_TEMPERATURE": np.float64,
          "SHELTER_TEMPERATURE_F": str,
          "QA_CODE": str}
for f in files:
    df = pd.read_csv(f, index_col=None, header=0, dtype=dtypes)
    df_list.append(df)

df = pd.concat(df_list, axis=0, ignore_index=True)

df.shape

(8443813, 30)

In [2]:
# This uses 2.7 to 3 gb in RAM
df = pd.read_parquet("files/metdata_combined.snappy.parquet",
                     engine="pyarrow",
                     dtype_backend="pyarrow")
df.shape

(8443813, 31)

1
